In [1]:
# Dependencies
import json
import requests 
from pprint import pprint
import pandas as pd
from pandas import json_normalize
import urllib
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
from datetime import datetime, date

In [2]:
# Specify the URL
url = "https://gbfs.citibikenyc.com/gbfs/es/station_status.json"

# Make request and store response
response = requests.get(url)

# Print status code
print(response.status_code)

200


In [3]:
r = requests.get('https://gbfs.citibikenyc.com/gbfs/en/station_status.json')
df = json_normalize(r.json()['data']['stations'])
df.head()

,legacy_id,station_status,station_id,num_bikes_available,is_renting,num_docks_disabled,is_returning,eightd_has_available_keys,num_ebikes_available,last_reported,num_bikes_disabled,is_installed,num_docks_available
0,72,active,72,5,1,0,1,False,1,1596472213,2,1,48
1,79,active,79,33,1,0,1,False,0,1596469694,0,1,0
2,82,active,82,22,1,0,1,False,0,1596470873,2,1,3
3,83,active,83,38,1,0,1,False,0,1596471155,1,1,23
4,116,active,116,8,1,0,1,False,1,1596472099,1,1,41


In [4]:
df2 = df.rename(columns={"num_bikes_available":"bikes_available","num_bikes_disabled":"bikes_disabled","num_docks_available":"docks_available","num_docks_disabled":"docks_disabled","last_reported":"time_reported"})

In [5]:
df2.head()

,legacy_id,station_status,station_id,bikes_available,is_renting,docks_disabled,is_returning,eightd_has_available_keys,num_ebikes_available,time_reported,bikes_disabled,is_installed,docks_available
0,72,active,72,5,1,0,1,False,1,1596472213,2,1,48
1,79,active,79,33,1,0,1,False,0,1596469694,0,1,0
2,82,active,82,22,1,0,1,False,0,1596470873,2,1,3
3,83,active,83,38,1,0,1,False,0,1596471155,1,1,23
4,116,active,116,8,1,0,1,False,1,1596472099,1,1,41


In [6]:
bike_df = df2[["station_id","bikes_available","bikes_disabled","docks_available","docks_disabled","time_reported"]]

In [7]:
bike_df.head()

,station_id,bikes_available,bikes_disabled,docks_available,docks_disabled,time_reported
0,72,5,2,48,0,1596472213
1,79,33,0,0,0,1596469694
2,82,22,2,3,0,1596470873
3,83,38,1,23,0,1596471155
4,116,8,1,41,0,1596472099


In [8]:

last_reported = datetime.fromtimestamp(bike_df.loc[0,"time_reported"])

In [9]:
last_reported

datetime.datetime(2020, 8, 3, 9, 30, 13)

In [10]:
def epoch_to_readable(xepoch):
    
    return datetime.fromtimestamp(xepoch)

bike_df['time_reported'] = bike_df.apply(lambda x: epoch_to_readable(x['time_reported']), axis = 1 )

<ipython-input-10-5919d25c1233>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bike_df['time_reported'] = bike_df.apply(lambda x: epoch_to_readable(x['time_reported']), axis = 1 )


In [11]:
bike_df.head()

,station_id,bikes_available,bikes_disabled,docks_available,docks_disabled,time_reported
0,72,5,2,48,0,2020-08-03 09:30:13
1,79,33,0,0,0,2020-08-03 08:48:14
2,82,22,2,3,0,2020-08-03 09:07:53
3,83,38,1,23,0,2020-08-03 09:12:35
4,116,8,1,41,0,2020-08-03 09:28:19


In [12]:
db_connection_string = "postgres:jaigurudev@localhost:5432/FinalProject"
engine = create_engine(f'postgresql://{db_connection_string}')

In [13]:
engine.table_names()

['bikes']

In [14]:
bike_df.to_sql(name = 'bikes',con = engine , if_exists = 'append',index = False)

In [15]:
pd.read_sql_query('select * from bikes', con=engine).head()

,station_id,bikes_available,bikes_disabled,docks_available,docks_disabled,time_reported
0,72,5,2,48,0,2020-08-03 09:30:13
1,79,33,0,0,0,2020-08-03 08:48:14
2,82,22,2,3,0,2020-08-03 09:07:53
3,83,38,1,23,0,2020-08-03 09:12:35
4,116,8,1,41,0,2020-08-03 09:28:19
